In [1]:
import pathlib
import os
import tensorflow as tf

import tfod
import bbox_prep
import detector

#### Detection Constants

In [2]:
# Path to base folder 
MAIN_FOLDER = pathlib.Path("./Main_Folder")

''' Detection Constants '''
# Path to label map
LABEL_PATH = pathlib.Path("./Tensorflow/workspace/training_demo/annotations/label_map.pbtxt")
# Path to pipeline file
CONFIG_PATH = pathlib.Path("./Tensorflow/workspace/training_demo/models/bbox_efficientdet1_v2/pipeline.config")
# Path to checkpoint file
CKPT_PATH = "./Tensorflow/workspace/training_demo/models/bbox_efficientdet1_v2/ckpt-21".encode('utf-8')
# Path to image folder. Do for pos and neg seperately
IMAGE_PATH = MAIN_FOLDER / "images"
# Path to the csv to save bboxes in
BBOX_CSV_NAME = IMAGE_PATH / "bboxes_ckpt21.csv"
# Resize all images to this before detection. Is set manually to have uniform images
PREF_SIZE = (1500, 1500) 

''' Bbox preperation constants '''
CONTRAST_FACTORS = [1.3, 1.5, 1.7]
CUT_IMAGE_SIZE = (220, 220)
CUT_SAVE_ADDRESS = IMAGE_PATH / "noCrownCrop"    # address to save croppped images

''' Classification Constants '''
# Seperate left and right side images to perform classification on them. Choosing contrast1.5 images is suggested. 
LEFT_IMAGES_PATH = MAIN_FOLDER / "left"            
RIGHT_IMAGES_PATH = MAIN_FOLDER / "right"
MODEL_LOAD_FOLDER = "model path"
MODEL_ADDRESS = MODEL_LOAD_FOLDER / "model name"   # Set the model name
BATCH_SIZE = 256

#### Tensorflow Object Detection, efficientdet_d1_coco17_tpu-32

In [3]:
image_list = list(IMAGE_PATH.glob("./*.png"))
tfod_instance = tfod.Tfod_User(CONFIG_PATH, CKPT_PATH, LABEL_PATH, image_list, PREF_SIZE, BBOX_CSV_NAME)
tfod_instance.model_loader()
tfod_instance.detection_batched()

tf.keras.backend.clear_session()
del tfod_instance

Detection of 0 files done, 20 remaining.
Detection of 5 files done, 15 remaining.
Detection of 10 files done, 10 remaining.
BBox Detection of 20 files finished.


#### Boundig Box Preparation to cut images and save them in a folder

In [ ]:
image_list = list(IMAGE_PATH.glob("./*.png"))
if not os.path.isdir(CUT_SAVE_ADDRESS):
    os.mkdir(CUT_SAVE_ADDRESS)

bbox_instance = bbox_prep.Bbox_User(image_list, BBOX_CSV_NAME, IMAGE_PATH, CUT_SAVE_ADDRESS, CONTRAST_FACTORS, prefix="c", size=CUT_IMAGE_SIZE)
bbox_instance.auto_handler()  

#### Classification: Predict and save result in a csv file

In [ ]:
left_images = list(LEFT_IMAGES_PATH.glob("./*.png"))
right_images = list(RIGHT_IMAGES_PATH.glob("./*.png"))
if len(left_images):
    result_csv_left = "./Classification Result Left.csv"
    classifier_instance = detector.Model_Handler(MODEL_ADDRESS, left_images, result_csv_left, (220, 220), "left", BATCH_SIZE)
    classifier_instance.auto_handler()
if len(right_images):
    result_csv_right = "./Classification Result Right.csv"
    classifier_instance = detector.Model_Handler(MODEL_ADDRESS, right_images, result_csv_right, (220, 220), "right", BATCH_SIZE)
    classifier_instance.auto_handler()